In [1]:
pip install streamlit openai crewai crewai-tools

  Using cached streamlit-1.42.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached openai-1.61.1-py3-none-any.whl.metadata (27 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-5.5.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cac

In [3]:
#pip install openai


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install yfinance

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------- ----------------------------- 0.8/3.0 MB 6.6 MB/s eta 0:00:01
     ------------------------ --------------- 1.8/3.0 MB 6.7 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 5.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------------- -------------------------- 1.3/3.8 MB 5.6 MB/s eta 0:00:01
   --------------------------- ------------ 2.6/3.8 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 6.0 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl 

In [10]:
import re
import json
import os
import yfinance as yf
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import CodeInterpreterTool, FileReadTool
from dotenv import load_dotenv

load_dotenv()

True

In [12]:

from pydantic import BaseModel, Field

class QueryAnalysisOutput(BaseModel):
    """Structured output for the query analysis task."""
    symbol: str = Field(..., description="Stock ticker symbol (e.g., TSLA, AAPL).")
    timeframe: str = Field(..., description="Time period (e.g., '1d', '1mo', '1y').")
    action: str = Field(..., description="Action to be performed (e.g., 'fetch', 'plot').")


In [14]:
llm = LLM(
    model="sambanova/DeepSeek-R1-Distill-Llama-70B",
    temperature=0.7
)

query_parser_agent = Agent(
    role="Stock Data Analyst",
    goal="Extract stock details and fetch required data from this user query: {query}.",
    backstory="You are a financial analyst specializing in stock market data retrieval.",
    llm=llm,
    verbose=True,
    memory=True,
)

query_parsing_task = Task(
    description="Analyze the user query and extract stock details.",
    expected_output="A dictionary with keys: 'symbol', 'timeframe', 'action'.",
    output_pydantic=QueryAnalysisOutput,
    agent=query_parser_agent,
)



# 2) Code writer agent
code_writer_agent = Agent(
    role="Senior Python Developer",
    goal="Write Python code to visualize stock data.",
    backstory="""You are a Senior Python developer specializing in stock market data visualization. 
                 You are also a Pandas, Matplotlib and yfinance library expert.
                 You are skilled at writing production-ready Python code""",
    llm=llm,
    verbose=True,
)

code_writer_task = Task(
    description="""Write Python code to visualize stock data based on the inputs from the stock analyst
                   where you would find stock symbol, timeframe and action.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_writer_agent,
)

# 3) Code interpreter agent (uses code interpreter tool)

code_interpreter_tool = CodeInterpreterTool()

code_execution_agent = Agent(
    role="Senior Code Execution Expert",
    goal="Review the generated Python code by code writer agent.",
    backstory="You are a code execution expert. You review Python code but do not execute it.",
    tools=[],  # Remove the CodeInterpreterTool
    allow_code_execution=False,  # Disable execution
    llm=llm,
    verbose=True,
)

code_execution_task = Task(
    description="""Review and execute the generated Python code by code writer agent to visualize stock data.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_execution_agent,
)


In [16]:
### --- CREW SETUP --- ###

crew = Crew(
    agents=[query_parser_agent, code_writer_agent, code_execution_agent],
    tasks=[query_parsing_task, code_writer_task, code_execution_task],
    process=Process.sequential
)

# Run the crew with an example query
result = crew.kickoff(inputs={"query": "Plot YTD stock gain of Tesla"})

2025-02-11 11:02:59,141 - 8688 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Stock Data Analyst
## Task: Analyze the user query and extract stock details.


# Agent: Stock Data Analyst
## Final Answer: 
{
  "symbol": "TSLA",
  "timeframe": "YTD",
  "action": "plot"
}


# Agent: Senior Python Developer
## Task: Write Python code to visualize stock data based on the inputs from the stock analyst
                   where you would find stock symbol, timeframe and action.


# Agent: Senior Python Developer
## Final Answer: 
Here is a complete Python script that visualizes stock data for TSLA (Tesla) using the specified timeframe (YTD) and performs plotting. The script uses Pandas, Matplotlib, and yfinance to fetch and visualize the stock data.

```python
# stock_visualizer.py
"""
This script visualizes stock data for a given symbol using yfinance and Matplotlib.
It plots the stock's price action and volume over the specified timeframe.
"""

import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd

def visualize_stock(symbol: str, timeframe